In [ ]:
import pandas as pd 
from dataclasses import dataclass
from typing import List
from datetime import datetime

In [ ]:
df_nov=pd.read_excel(r"..\Data\LineC_SessionData-individual rats_SampleData.xlsx")
df_march=pd.read_excel(r"..\Data\Snifftime data_March15.2019_example_withIndividualSniffs.xlsx")
df_sept=pd.read_excel(r"..\Data\AutomatedCage_Sept2023-24_raw data.xlsx")